In [3]:
import numpy as np
import cv2 as cv
import glob
import matplotlib.pyplot as plt

In [4]:
CELL_LENGTH = 21.34
ROWS = 9
COLS = 6

In [7]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((COLS * ROWS, 3), np.float32)
objp[:,:2] = np.mgrid[0:ROWS,0:COLS].T.reshape(-1,2) * CELL_LENGTH
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('images/webcam/*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (ROWS, COLS), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (ROWS, COLS), corners2, ret)
        plt.figure(figsize = (20,15))
        # plt.imshow(img)
        # plt.show()
        # cv.imshow('img', img)
        # cv.waitKey(0)



cv.destroyAllWindows()

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

<Figure size 2000x1500 with 0 Axes>

In [6]:
corners

array([[[1070.0829 ,  303.98282]],

       [[1083.5582 ,  342.51672]],

       [[1098.9581 ,  386.50488]],

       [[1115.4678 ,  435.23123]],

       [[1134.4766 ,  487.87766]],

       [[1155.2477 ,  547.5642 ]],

       [[1180.5076 ,  615.8313 ]],

       [[1207.5593 ,  693.89215]],

       [[1241.0052 ,  784.72595]],

       [[ 986.333  ,  297.24985]],

       [[ 996.00775,  336.54172]],

       [[1006.3091 ,  381.00522]],

       [[1018.4272 ,  428.71594]],

       [[1032.1095 ,  479.86832]],

       [[1047.2284 ,  539.481  ]],

       [[1063.7987 ,  608.13654]],

       [[1083.876  ,  687.5284 ]],

       [[1106.1725 ,  778.9201 ]],

       [[ 901.3254 ,  290.24524]],

       [[ 906.60693,  329.6677 ]],

       [[ 912.6213 ,  373.21457]],

       [[ 919.3409 ,  421.08957]],

       [[ 927.0762 ,  472.21973]],

       [[ 935.48694,  531.2523 ]],

       [[ 945.3867 ,  600.7759 ]],

       [[ 956.3048 ,  679.5139 ]],

       [[ 969.3057 ,  772.9911 ]],

       [[ 814.63477,  283.37

In [22]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
img = cv.imread('images/webcam/00.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True

In [29]:
# get coordinates of an image by clicking on it. Store the coordinates in a list
coordinates = []
def click_event(event, x, y, flags, params):
    if event == cv.EVENT_LBUTTONDOWN:
        print(x, ' ', y)
        coordinates.append((x, y))
        font = cv.FONT_HERSHEY_SIMPLEX
        cv.putText(img, str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv.imshow('image', img)
    if event == cv.EVENT_RBUTTONDOWN:
        cv.destroyAllWindows()

img = cv.imread('images/webcam/00.jpg')
cv.imshow('image', img)
cv.setMouseCallback('image', click_event)
cv.waitKey(0)
cv.destroyAllWindows()
